In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito_data['fecha_2da_insp'] = datetime.strptime(credito_data['fecha_2da_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    # query_propiedades = {'query': 'select * from dataset_282968 where cre_ref_prop = \'{cre_ref_prop}\''.format(cre_ref_prop=credito.amigo_id)}
    query_propiedades = {'query': 'select prop.propiedad, prop.amigo_id from dataset_307562 prop inner join dataset_307563 lotes on lotes.prop_ref_lote=prop.amigo_id where insp_ref_prop=\'{cre_ref_prop}\' and (lotes.estado_de_siembra=\'CAÑA NACIDA\' or lotes.estado_de_siembra=\'CAÑA NO NACIDA\') group by prop.propiedad, prop.amigo_id'.format(cre_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)    
    
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\' and (estado_de_siembra=\'CAÑA NACIDA\' or estado_de_siembra=\'CAÑA NO NACIDA\')'.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_48961 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate(ruta + "/templates/tpl_inf_credito_2da.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open(ruta + '/fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_2da_insp == 'Juan Pablo Jaldin Duran':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_jaldin.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_rogelio.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    file_name = file_name = '_' + cod_ca + '_CRS_' + credito.fecha_2da_insp.replace('/', '-') + '_' + nom_ca + '_' + str(credito.id)
    doc.save(ruta + '/informes/' + file_name + '.docx')

In [5]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta_plano = ruta + '/planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta_plano, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta_plano

In [6]:
semana = 'S28'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_2da = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [7]:
data

[{'cod': '41529', 'nom': ' ACUÑA CANO VICTORIANO'},
 {'cod': '42148', 'nom': 'ESPINOZA ESTHER AGUILAR DE'},
 {'cod': '15195', 'nom': 'VEIZAGA ENRIQUEZ GUSTAVO'},
 {'cod': '42025', 'nom': 'TRUJILLO CONDORI JAIME '},
 {'cod': '17678', 'nom': 'ESPINOZA JUANA MAIDA DE '},
 {'cod': '9722', 'nom': 'OLGUIN FLORES MIGUELINA '},
 {'cod': '11227', 'nom': 'QUIROZ OLGUIN DENICE'},
 {'cod': '41555', 'nom': 'FRANCO GUZMAN CARLA DANIELA '},
 {'cod': '1346', 'nom': 'BORDA POMA OLGA '},
 {'cod': '5006', 'nom': 'GARCIA MARISCAL MILTON FELIX'},
 {'cod': '2816', 'nom': 'DONOSO BLANCO ALBERTO'}]

In [8]:
# lista = [430, 40270, 13132, 874, 41966, 11798, 41544, 14993, 644, 1068, 13763, 2220, 1858, 2719]
lista = [i['cod'] for i in data]

In [9]:
lista

['41529',
 '42148',
 '15195',
 '42025',
 '17678',
 '9722',
 '11227',
 '41555',
 '1346',
 '5006',
 '2816']

In [10]:
#lista=['12998']

In [12]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L2.2 de2656bc2f29436ba01210eb5a90a50b
L2.3 d9508745495848049b64baaa82dc6ffa
L2.1 2b74d4e73d134bed9b57748afa09f0c9
L3.1 ca60a5f08aac4b66b6063ceb1aa0678c
====================== 41529
L3 0def8c91639b4a989c8d8c01bb194d70
L2 cb6fee66961a484da0e28f449211b486
L1 efd4ed5a098143b69995d841fbd85e8c
====================== 42148
L4.2 6f5fade9c72c493e836ec6754c668d80
====================== 15195
L5 54ec33bfcfcf4b3ea5471d14f4dc3dcb
L2 c1bea7feaa1e41c88b92542635b52a69
====================== 42025
L1.2 c6a03bc2d3c24eb1938d59848c0977ef
====================== 17678
L2 8e0337b5f1854d7dbe5a85aadfd2b4d0
L6 f5790f35656044dd95225094e315d681
L5 9da70bf95b3f47aeb6254c2805c30d35
L1.4 a0942db06f6c4d41ae21cc52e57020ae
L1.2 8ba654351744495187a44c5e3dc0c0b6
L8.5 2738fddcb57845d0a1b8a87aa6e45aa8
L7 99eef9b868564a698e53241737719cee
L1 eed7f88f465f4aedaee16efb3d6381b8
====================== 9722
L2.1 4906259a5149492fbe433cefe7b2448a
L3 d0fd2cf95a6c43d8b74d4f2515e17e42
L4 d0968d48567743dda7dbb2231e2c9bc2
L2.4 cdddd2b32c

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>